In [9]:
import pandas as pd
import numpy as np
import random as rn
import math




In [10]:
import math
import random as rn
import numpy as np
import pandas as pd

class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x) - 1, 0, -1):
            j = rn.randint(0, i - 1)
            x.iloc[i], x.iloc[j] = x.iloc[j], x.iloc[i]

    @staticmethod
    def normalization(x, names):
        for column in names:
            data = x.loc[:, column]
            max1 = max(data)
            min1 = min(data)
            for row in range(0, len(x), 1):
                xprim = (x.at[row, column] - min1) / (max1 - min1)
                x.at[row, column] = xprim

    @staticmethod
    def getRating(x):
        lis = [0] * len(x)
        i = 0
        for index, row in x.iterrows():
            lis[i] +=  math.sqrt(row['Views']) + 2 * row['Likes'] / (row['Views']+1) + (row['Comments'] * 0.5) / (row['Views'] +1)
            i+=1
        return lis

    @staticmethod
    def split(x, k):  # k = 0.7, 70% do treningowego
        splitPoint = int(len(x) * k)
        return x.iloc[0:splitPoint], x.iloc[splitPoint:]

    @staticmethod
    def getDistances(x, newObj, columnNames, function, power):
        lis = [0] * len(x)
        i = 0
        for index, row in x.iterrows():
            for column in columnNames:
                lis[i] +=(function(row[column], newObj[column], power))
            i+=1
        return lis

    @staticmethod
    def manhattan(first, second, n):
        return abs(first - second)
                
    @staticmethod
    def euclides(first, second, n):
        difference = math.pow(first, n) - math.pow(second, n)
        return difference ** 1/n
    
    

    @staticmethod
    def sort(x, lis):
        x['distance'] = lis
        return x.sort_values(by=['distance'])


    @staticmethod
    def isBanger(x, bar):
        lis = [0]*len(x)
        i = 0
        for index, row in x.iterrows():
            if row['Rating'] >= bar:
                lis[i] = 1
            i+=1
        return lis
            


    @staticmethod
    def NaiveBayes(x, sample, classCol, columnNames):
        classes = x[classCol].unique().tolist()
        
        res = {}
        for var in classes:
            res[var] = []
            for cl in columnNames:
                values = x.loc[x[classCol] == var, cl]
                mean = values.mean()
                sigm2 = values.std()**2
                if sigm2 == 0:
                    sigm2=0.00000001
                
                res[var].append(DataProcessing.gauss(sample[cl], mean, sigm2))
            res[var] = 1 / len(classes) * np.prod(res[var])
        return max(res, key=res.get)

    @staticmethod
    def bayes(x, sample, classCol, colNames):
        classes = x[classCol].unique().tolist()
        res = {}
        lis = []
        for i in classes:
            lis.append(x[x[classCol]==i])

        for var in lis:
            res[str(var[classCol][0])] = []
            for cl in colNames:
                values = x[cl]
                mean = values.mean()
                sigm2 = values.std()**2
                if sigm2 == 0:
                    sigm2=0.00000001
                
                res[str(var[classCol][0])].append(DataProcessing.gauss(sample[cl], mean, sigm2))
            res[str(var[classCol][0])] = 1 / len(classes) * np.prod(res[str(var[classCol][0])])
        return max(res, key=res.get)
               



    @staticmethod
    def gauss(x, mu, sigm2):
        return (1 / sigm2 * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu) / sigm2)**2)

    @staticmethod
    def useBayes(tS, vS):
        counter = 0
        for i in range(0, len(vS)):
            print(DataProcessing.NaiveBayes(tS, vS.iloc[i], 'variety'), vS.iloc[i]['variety'])
            if DataProcessing.NaiveBayes(tS, vS.iloc[i], 'variety') == vS.iloc[i]['variety']:
                counter += 1
        return counter / len(vS)


In [11]:
data = pd.read_csv('spotify-yt-fixed.csv')


In [12]:
data = data.sample(frac=1)
data.iloc[0]

Unnamed: 0                                               8130
Artist                                           Depeche Mode
Track               Never Let Me Down Again - Tsangarides Mix
Danceability                                            0.573
Energy                                                  0.936
Key                                                       0.0
Loudness                                               -4.195
Speechiness                                            0.0321
Acousticness                                            0.294
Instrumentalness                                       0.0327
Liveness                                                0.116
Valence                                                 0.784
Tempo                                                 106.103
Duration_ms                                          262000.0
Views                                                  3439.0
Likes                                                    48.0
Comments

In [13]:
DataProcessing.normalization(data,['Loudness', 'Tempo', 'Duration_ms', 'Views', 'Likes', 'Comments'] )
rating = DataProcessing.getRating(data)
data['Rating'] = rating
bar = data.sort_values('Rating', ascending=False).loc[2000, 'Rating']
data['Hit'] = DataProcessing.isBanger(data, bar)
data.drop(labels=['Views', 'Likes', 'Comments', 'Rating', 'Key'], inplace=True, axis=1)
data

,Unnamed: 0,Artist,Track,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Hit
7346,8130,Depeche Mode,Never Let Me Down Again - Tsangarides Mix,0.573,0.936,0.891565,0.0321,0.2940,0.03270,0.1160,0.784,0.435970,0.244634,0
7290,8074,TWICE,Feel Special,0.708,0.959,0.952132,0.0604,0.0296,0.00188,0.1070,0.644,0.529975,0.186247,1
889,958,Vishal-Shekhar,"Besharam Rang (From ""Pathaan"")",0.773,0.795,0.863115,0.0412,0.0587,0.00237,0.1540,0.649,0.476624,0.240900,1
6620,7326,Grupo 5,Quien Cura,0.731,0.786,0.911810,0.0275,0.2720,0.00000,0.2630,0.830,0.402774,0.191694,1
12649,13833,IU,Can't Love You Anymore (With OHHYUK),0.611,0.630,0.886816,0.3630,0.5660,0.00000,0.2370,0.660,0.269624,0.173923,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,2447,Banda El Recodo,Deja,0.417,0.465,0.831485,0.0550,0.5570,0.00000,0.1310,0.688,0.697899,0.133098,0
12158,13298,Sean Kingston,Dumb Love,0.758,0.646,0.902864,0.0721,0.3560,0.00000,0.4730,0.877,0.526067,0.164967,0
12535,13708,The Script,Run Through Walls,0.337,0.660,0.903246,0.0324,0.2260,0.00000,0.7440,0.490,0.652762,0.186063,1
18643,20187,ITZY,Boys Like You,0.836,0.903,0.890420,0.1050,0.0318,0.00000,0.0688,0.882,0.525981,0.203727,1


max_row = df['A'].idxmax()

In [14]:
DataProcessing.shuffle(data)
ts, vs = DataProcessing.split(data, 0.9)




In [ ]:
k_accuracy_mahnattan = {100:0, 50:0, 35:0, 20:0, 5:0, 10:0}
k_accuracy_euclides = {100:0, 50:0, 35:0, 20:0, 5:0, 10:0}
k_accuracy_minkowski = {100:0, 50:0, 35:0, 20:0, 5:0, 10:0}

In [27]:
stac = ['Danceability', 'Energy', 'Loudness', 'Acousticness', 'Instrumentalness',
 'Liveness', 'Valence', 'Tempo', 'Duration_ms']
lenvs = len(vs)
for element in k_accuracy_mahnattan:
    counter = 0
    for i, r in vs.iterrows():
        lis = DataProcessing.getDistances(ts, r, stac, DataProcessing.manhattan, 2)
        recs = DataProcessing.sort(ts, lis)
        if DataProcessing.NaiveBayes(recs[:int(element)], r, 'Hit', stac) == r['Hit']:
            counter+=1
    k_accuracy_mahnattan[element] = counter / lenvs



C:\Users\barte\AppData\Local\Temp\ipykernel_16784\412613330.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['distance'] = lis


In [ ]:
counter = 0
smallerts = ts[:len(ts)//2]
smallervs = vs[:len(vs)//2]
stac = ['Danceability', 'Energy', 'Loudness', 'Acousticness', 'Instrumentalness',
'Liveness', 'Valence', 'Tempo', 'Duration_ms']
for i, r in smallervs.iterrows():
    lis = DataProcessing.getDistances(smallerts, r, stac, DataProcessing.manhattan, 2)
    recs = DataProcessing.sort(smallerts, lis)
    f50 = recs[:50]
    if DataProcessing.NaiveBayes(f50, r, 'Hit', stac) == r['Hit']:
        counter+=1
k_accuracy["50"] = counter/len(smallervs)
k_accuracy['50']

C:\Users\barte\AppData\Local\Temp\ipykernel_16784\412613330.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['distance'] = lis


KeyboardInterrupt: 

In [ ]:
counter = 0
smallerts = ts[:len(ts)//2]
smallervs = vs[:len(vs)//2]
stac = ['Danceability', 'Energy', 'Loudness', 'Acousticness', 'Instrumentalness',
'Liveness', 'Valence', 'Tempo', 'Duration_ms']
for i, r in smallervs.iterrows():
    lis = DataProcessing.getDistances(smallerts, r, stac, DataProcessing.manhattan, 2)
    recs = DataProcessing.sort(smallerts, lis)
    f25 = recs[:25]
    if DataProcessing.NaiveBayes(f25, r, 'Hit', stac) == r['Hit']:
        counter+=1
k_accuracy["25"] = counter/len(smallervs)
k_accuracy['25']

C:\Users\barte\AppData\Local\Temp\ipykernel_7844\1236735400.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['distance'] = lis


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.6070686070686071